In [1]:
import pandas as pd



In [10]:
def process_and_match(file1_path, file2_path, output_path):
    # 첫 번째 파일 읽기
    df1 = pd.read_csv(file1_path)

    # 두 번째 파일 읽기
    df2 = pd.read_csv(file2_path)

    # 결과를 저장할 데이터프레임 생성
    result_rows = []

    # 첫 번째 파일의 각 행을 순회하며 measure 범위와 feature를 추출
    for _, row in df1.iterrows():
        name = row['name']
        feature = row['important_feature']
        start = row['interval_start']
        end = row['interval_end']

        for measure in range(start, end + 1):
            # 두 번째 파일에서 일치하는 measure와 feature 필터링
            match_df = df2[(df2['measure'] == measure) & (df2['feature'] == feature)]

            if not match_df.empty:
                # 일치하는 데이터가 있는 경우
                for _, match_row in match_df.iterrows():
                    result_rows.append([
                        match_row['measure'],
                        match_row['feature'],
                        match_row['feature_value']
                    ])
            else:
                # 일치하는 데이터가 없는 경우
                result_rows.append([measure, feature, 'null'])

    # 결과 데이터프레임으로 변환
    result_df = pd.DataFrame(result_rows, columns=['measure', 'feature', 'feature_value'])
    print(result_df)
    
    # critical_score 계산
    critical_score_df = result_df.groupby(['measure', 'feature', 'feature_value']).size().reset_index(name='critical_score')

    # 결과를 CSV 파일로 저장
    critical_score_df.to_csv(output_path, index=False)
    print(f'Result saved to {output_path}')

In [11]:

# 사용 예시
process_and_match('c_score.csv', 'overlapped2.csv', 'test_result.csv')

      measure  feature feature_value
0        39.0      1.0      0.666667
1        40.0      1.0      0.666667
2        41.0      1.0      0.666667
3        42.0      1.0      0.666667
4        40.0      1.0      0.666667
...       ...      ...           ...
1046     43.0     18.0             1
1047     41.0     18.0             1
1048     42.0     18.0             1
1049     43.0     18.0             1
1050     44.0     18.0             1

[1051 rows x 3 columns]
Result saved to test_result.csv


In [12]:
def merge_csv(file1_path, file2_path, output_path):
    # 두 CSV 파일 읽기
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)

    # 두 번째 파일에서 critical_score가 2 이상인 행만 필터링
    df2_filtered = df2[df2['critical_score'] >= 2]

    # 두 DataFrame을 병합 (중복 제거)
    merged_df = pd.concat([df1, df2_filtered]).drop_duplicates(subset=['measure', 'feature'], keep='first')

    # 결과를 CSV로 저장
    merged_df.to_csv(output_path, index=False)
    print(f'Result saved to {output_path}')

In [13]:

# 사용 예시
merge_csv('test_result.csv', 'overlapped2.csv', 'test_result2.csv')


Result saved to test_result2.csv


In [14]:
def convert_csv(pid_csv, performance_csv, output_csv):
    # 파일 읽기
    pid_df = pd.read_csv(pid_csv)
    performance_df = pd.read_csv(performance_csv)
    
    # midi_path와 pid를 딕셔너리로 매핑
    path_to_pid = dict(zip(pid_df['midi_path'], pid_df['pid']))
    
    # performance_id를 pid로 변환
    performance_df['pid'] = performance_df['performance_id'].map(path_to_pid)
    
    # 변환 과정: measure, feature, feature_value로 변경
    performance_df['measure'] = performance_df['bar'] + 1
    performance_df['feature'] = performance_df['feature_idx'] + 1
    performance_df['feature_value'] = performance_df['value']
    
    # 필요한 열 선택 및 순서 지정
    result_df = performance_df[['pid', 'measure', 'feature', 'feature_value']]
    
    # 결과 CSV 파일 저장
    result_df.to_csv(output_csv, index=False)
    print(f'변환된 데이터가 {output_csv}로 저장되었습니다.')

In [15]:
# 사용 예시
convert_csv('pidname.csv', 'concatenated_v1.csv', 'test_result3.csv')

변환된 데이터가 test_result3.csv로 저장되었습니다.


In [25]:
def compare_csv(performance_csv, guideline_csv, output_csv):
    # CSV 파일 읽기
    performance_df = pd.read_csv(performance_csv)
    guideline_df = pd.read_csv(guideline_csv)
    
    # pid가 45에서 60 사이인 행만 필터링
    filtered_df = performance_df[(performance_df['pid'] >= 45) & (performance_df['pid'] <= 60)]
    
    # measure와 feature를 기준으로 병합 (right join)
    merged_df = pd.merge(filtered_df, guideline_df, on=['measure', 'feature'], how='right')
    print(merged_df)
    
    # difference 계산
    merged_df['difference'] = merged_df['feature_value_x'] - merged_df['feature_value_y']
    
    # 열 이름 변경
    merged_df = merged_df.rename(columns={'feature_value_x': 'feature_value','feature_value_y': 'guideline_value', 'critical_score': 'critical_score'})
    
    # 필요한 열만 선택
    result_df = merged_df[['pid', 'measure', 'feature', 'feature_value', 'guideline_value', 'difference', 'critical_score']]
    
    # 결과 저장
    result_df.to_csv(output_csv, index=False)
    print(f'비교 결과가 {output_csv}로 저장되었습니다.')

In [26]:
# 사용 예시
compare_csv('test_result3.csv', 'test_result2.csv', 'test_result4.csv')

       pid  measure  feature  feature_value_x  feature_value_y  critical_score
0     54.0        7        4         0.553663         0.666667               1
1     53.0        7        4         0.557523         0.666667               1
2     55.0        7        4         0.528695         0.666667               1
3     59.0        7        4         0.562487         0.666667               1
4     60.0        7        4         0.568413         0.666667               1
...    ...      ...      ...              ...              ...             ...
2852  54.0       70       19         0.494542         1.000000               3
2853   NaN       70       23              NaN         1.000000               3
2854   NaN       77       23              NaN         1.000000               2
2855   NaN       78       23              NaN         1.000000               2
2856   NaN       79       20              NaN         0.033333               2

[2857 rows x 6 columns]
비교 결과가 test_result4.csv로 저장